``plot_trace``関数でワーニングを出さないため次のコマンド実行が必要です。

In [ ]:
!pip install --upgrade numba | tail -n 1

##  6.2 ベイス回帰モデルによる効果検証
参照リンク　(PyMCチュートリアル)　https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#case-study-1-educational-outcomes-for-hearing-impaired-children

チュートリアルでは正則化を含んだ複雑なモデルを作っているが、簡略化して単純な線形回帰モデルを作り、各説明変数の影響度を考察する

<a href="https://colab.research.google.com/github/makaishi2/python_bayes_intro/blob/main/notebooks/6_2_%E3%83%99%E3%82%A4%E3%82%B9%E5%9B%9E%E5%B8%B0%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AB%E3%82%88%E3%82%8B%E5%8A%B9%E6%9E%9C%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 共通処理

In [ ]:
%matplotlib inline
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np
# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt
# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib
# pandas用ライブラリ
import pandas as pd
# データフレーム表示用関数
from IPython.display import display
# seaborn
import seaborn as sns
# 表示オプション調整
# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)
# 方眼表示ON
plt.rcParams['axes.grid'] = True
# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

In [ ]:
import pymc as pm
import arviz as az

print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArViz v{az.__version__}")

### 6.2.1 問題設定

　本節では、Listening and Spoken Language Data Repository (LSL-DR)で提供されているデータを用いて、ベイス推論により線形回帰モデルを構築します。

　LSL-DRは難聴の子供の音声言語スキルの発達を支援する専門教育プログラムにおける国際的なデータリポジトリです。4か国の48のプログラムで、5,748人の難聴の子供の情報を収集し、彼らの音声言語の発達に影響を与える要因を調査するデータを収集しています。


　このデータセットには、以下のような項目が含まれています。

| 項目名        | 説明                                           | 項目値      |
| ------------- | ---------------------------------------------- | ----------- |
| score         | 能力テストのスコア(目的変数)                   | 0-144の整数 |
| male          | 性別                                           | 1/0         |
| siblings      | 世帯内の兄弟の数                               | 非負整数値  |
| family_inv    | 家族の関与の指標                               | 0-4の整数値 |
| non_english   | 家庭での主な言語が英語ではないか               | True/False  |
| prev_disab    | 以前の障害の存在                               | 1/0         |
| age_test      | テスト時の年齢                                 | 48-59の整数 |
| non_severe_hl | 重度の難聴ではないか                           | 1/0         |
| mother_hs     | 被験者の母親が高校卒業以上の学歴を持っているか | 1/0         |
| early_ident   | 聴覚障害が生後3か月までに特定されたか          | True/False  |
| non_white     | 非白人                                         | True/False  |


　目的変数は、学習ドメインにおいて標準化されたテストのスコア の1つを用いています(score)。

### 6.2.2 データ読み込み

In [ ]:
# LSL-DR データ読み込み
df = pd.read_csv(pm.get_data('test_scores.csv'), index_col=0)

# 結果確認
display(df.head())

### 6.2.3 データ確認

#### 目的変数scoreの分布

In [ ]:
bins = np.arange(0, 150, 10)
fig, ax = plt.subplots()
df['score'].hist(bins=bins, align='left')
plt.setp(ax.get_xticklabels(), rotation=90)
plt.title('目的変数scoreスコアの分布')
plt.xticks(bins);

#### 統計情報の確認

In [ ]:
df.describe()

#### データ件数と欠損値の確認

In [ ]:
# データ件数の確認
print(f'データ件数 {len(df)}\n')

# 欠損値値の確認
print(df.isnull().sum())

### 6.2.4 データ加工

#### 欠損値除去

In [ ]:
# 欠損値除去
df1 = df.dropna().astype(float)

# データ件数の確認
print(f'データ件数 {len(df1)}')

#### 目的変数yと説明変数Xに分離

In [ ]:
y = df1.pop("score")
X = df1.copy()

# Xの結果確認
display(X.head())

#### Xの正規化

In [ ]:
X -= X.mean()
X /= X.std()

# 結果確認
display(X.head())

#### ベイズモデル構築に必要な変数定義

In [ ]:
# データ件数とデータ項目数の設定
N, D = X.shape

# 項目名一覧をcolumnsに設定する
columns = X.columns.values

# 結果確認
print(f'N: {N} (データ件数)\n')
print(f'D: {D} (説明変数項目数)\n')
print(f'項目名一覧: {columns}')

### 6.2.5 確率モデル定義

In [ ]:
# 説明変数リストをpredictorsとして定義
model1 = pm.Model(coords={'predictors': columns})

with model1:
    # Xは従来のベクトルが行列になる。転置していることに注意
    X_data = pm.ConstantData('X_data', X.T)

    # yが回帰モデルの目的変数
    y_data = pm.ConstantData('y_data', y)

    # 単回帰のときスカラーだったalphaは重回帰でベクトルになる
    # 要素数はpredictorsにより間接的に指定できる(上でcoordsパラメータを指定した効果)
    alpha = pm.Normal('alpha', mu=0.0, sigma=10.0, dims='predictors')

    # betaとepsilonは単回帰の時と同じ(パラメータ値の選定理由は本文で説明)
    beta = pm.Normal('beta', mu=100.0, sigma=25.0)
    epsilon = pm.HalfNormal('epsilon', sigma=25.0)

    # muの計算では、単回帰のときのかけ算が内積に変わっている
    mu = pm.Deterministic('mu', alpha @ X_data + beta)

    # 正規分布の定義は5.2節の単回帰と同じ
    obs = pm.Normal('obs', mu=mu, sigma=epsilon, observed=y_data)

g = pm.model_to_graphviz(model1)
display(g)

### 6.2.6 サンプリングと結果分析

#### サンプリングとplot_trace関数による」結果分析

In [ ]:
with model1:
    idata1 = pm.sample(random_seed=42, target_accept=0.95)

# plot_trace関数で推論結果の確認
az.plot_trace(idata1, var_names=['alpha', 'beta', 'epsilon'], compact=False)
plt.tight_layout();

#### サンプリング結果の統計情報取得

In [ ]:
summary1 = az.summary(idata1, var_names=['alpha'])
display(summary1)

#### plot_forest関数で各項目の寄与度の確認

In [ ]:
az.plot_forest(idata1, combined=True, var_names=['alpha']);

#### plot_forest関数　combinedオプションを指定しない場合

In [ ]:
az.plot_forest(idata1, var_names=['alpha']);

### コラム　チュートリアルの確率モデル

#### 確率モデル定義

In [ ]:
# D0の定義
D0 = int(D / 2)

# 説明変数リストをpredictorsとして定義
model2 = pm.Model(coords={'predictors': columns})

with model2:

    # Xは従来のベクトルが行列になる。転置していることに注意
    X_data = pm.ConstantData('X_data', X.T)

    # yが回帰モデルの目的変数
    y_data = pm.ConstantData('y_data', y)

    # 誤差の分布 sigma -> epsilon 文字の置き換えのみ
    epsilon = pm.HalfNormal('epsilon', sigma=25.0)

    # 一次関数の係数の分布　beta -> alpha 定義内容も全面的に変更

    # 事前分布の全体的な縮小
    tau = pm.HalfStudentT("tau", 2, D0 / (D - D0) * epsilon / np.sqrt(N))

    # 事前分布の局所的な縮小
    lam = pm.HalfStudentT("lam", 2, dims="predictors")
    c2 = pm.InverseGamma("c2", 1, 0.1)
    z = pm.Normal("z", 0.0, 1.0, dims="predictors")

    alpha = pm.Deterministic(
        "alpha", z * tau * lam * pm.math.sqrt(
        c2 / (c2 + tau**2 * lam**2)), dims="predictors")

    # 定数項　beta0 -> beta　文字の置き換えのみ
    beta = pm.Normal("beta",  mu=100.0, sigma=25.0)

    # 正規分布の平均　mu np.dotを@に変えたがロジックは同じ
    mu = pm.Deterministic('mu', alpha @ X_data + beta)

    # 観測値の分布　scores -> obs 文字の置き換えのみ
    obs = pm.Normal("obs", mu, epsilon, observed=y_data)

# 確率モデル可視化
g = pm.model_to_graphviz(model2)
display(g)

#### サンプリングと推論結果の確認

In [ ]:
with model2:
    idata2 = pm.sample(random_seed=42, target_accept=0.95)

# plot_trace関数で推論結果の確認
az.plot_trace(idata2, var_names=['alpha', 'beta', 'epsilon'], compact=False)
plt.tight_layout();

#### plot_forest関数で各項目の寄与度の確認

In [ ]:
az.plot_forest(idata2, combined=True, var_names=['alpha']);

#### サンプリング結果の統計分析

In [ ]:
summary2 = az.summary(idata2, var_names=['alpha'])
display(summary2)

#### バージョンの確認

In [ ]:
!pip install watermark | tail -n 1
%load_ext watermark
%watermark --iversions